<a href="https://colab.research.google.com/github/lucasBertola/Connect-4-Gym-env-Reinforcement-learning/blob/main/exemples/Self_play_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies

In [ ]:
%%capture
!pip install gymnasium-connect-four==1.3.1 stable-baselines3==2.0.0

# Self training

In [ ]:
%%capture
from connect_four_gymnasium import ConnectFourEnv
from connect_four_gymnasium.players import ModelPlayer
from stable_baselines3 import PPO

env = ConnectFourEnv()
model = PPO("MlpPolicy", env, verbose=1)
opponent =  ModelPlayer(model,name="yourself")
env.change_opponent(opponent)

In [ ]:
model.learn(total_timesteps=10000)

# Test your model

In [ ]:
from connect_four_gymnasium.tools import EloLeaderboard
from connect_four_gymnasium.players import ModelPlayer

print('Your elo: ',EloLeaderboard().get_elo(opponent, num_matches=250))

# Bonus : Learning from the games of both opponents

As you may have noticed, we have just played against ourselves to learn, but we are greatly underutilizing the playing time. Indeed, we learn from player 1, but not from player 2 (despite it being the same model as player 1).

In order to double the learning base without spending more time on simulation, it can be useful to learn from the perspective of both opponents. Here's how to retrieve the games. (Please note that PPO does not allow learning from games provided as input, so you will need to use another library (or your own) than Stable Baselines to take advantage of these games).

In [ ]:
import numpy as np
env = ConnectFourEnv() #without opponent
AllGamesMoves = []
GameToPlay = 100
for i in range(GameToPlay):
  gameIsFinish = False
  moves = []
  obs , _=  env.reset()
  while gameIsFinish == False:
    action, _state = model.predict(obs)
    moves.append((np.copy(obs),int(action), 0)) # we will update 0 later, it's the final score for this player
    obs, rewards, dones, truncated, info = env.step(action)
    if truncated or dones:
      # Update rewards for each move by iterating through the reversed moves list
      # If the index of the move is even or odd like the last move's index, assign the positive reward, otherwise assign the negative reward
      moves = [(move[0], move[1], rewards if idx % 2 == (len(moves) - 1) % 2 else -rewards) for idx, move in enumerate(reversed(moves))]
      gameIsFinish=True
      AllGamesMoves.extend(moves)

print(len(AllGamesMoves))
